#### This script trains the parameters for pure ML model of stomatal conductnace under Configuration-1.

In [ ]:
import numpy as np
# SW_model_LE: calculate the LE using Shutteleworth-Wallace model given rsc and other input variable
def SW_model_LE_np(auxi, rsc_pre):
    
    #---- SW-Model ----
    # --> LE = LE_e + LE_c = w_s*PM_s + w_c*PM_c
    # --> PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    # --> PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    # --> w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    # --> w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    # --> R_s = (delta + Psy)*ras + Psy*rss
    # --> R_c = (delta + Psy)*rac + Psy*rsc
    # --> R_a = (delta + Psy)*raa
    
    # --> A = Rn - G
    # --> A_s = Rns - G
    # --> Rns = Rn*exp(-Kr*LAI)
    #----- END ----

    Kr = 0.6
    A = auxi[25,:] - auxi[26,:]
    Rns = auxi[25,:]*np.exp(-Kr*auxi[27,:])
    A_s = Rns - auxi[26,:]
    delta = auxi[17,:]
    rho = auxi[18,:]
    Cp = auxi[20,:]
    Psy = auxi[19,:]
    VPD = auxi[28,:]/10   # Check the units (should be in kPa)
    ras = auxi[21,:]
    rac = auxi[22,:]
    raa = auxi[23,:]
    rss = auxi[16,:]
    PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    rsc = np.exp(rsc_pre)  # y = log(rsc)
    rsc = np.clip(rsc,1e-5,5000)
    PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    R_s = (delta + Psy)*ras + Psy*rss
    R_c = (delta + Psy)*rac + Psy*rsc
    R_a = (delta + Psy)*raa
    w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    
    T_pre = w_c*PM_c
    LE_pre = w_s*PM_s + w_c*PM_c
    
    return LE_pre

# SW_model_T: calculate the T (Transpiration) using Shutteleworth-Wallace model given rsc and other input variable
def SW_model_T_np(auxi, rsc_pre):
    
    #---- SW-Model ----
    # --> LE = LE_e + LE_c = w_s*PM_s + w_c*PM_c
    # --> PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    # --> PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    # --> w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    # --> w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    # --> R_s = (delta + Psy)*ras + Psy*rss
    # --> R_c = (delta + Psy)*rac + Psy*rsc
    # --> R_a = (delta + Psy)*raa
    
    # --> A = Rn - G
    # --> A_s = Rns - G
    # --> Rns = Rn*exp(-Kr*LAI)
    #----- END ----

    Kr = 0.6
    A = auxi[25,:] - auxi[26,:]
    Rns = auxi[25,:]*np.exp(-Kr*auxi[27,:])
    A_s = Rns - auxi[26,:]
    delta = auxi[17,:]
    rho = auxi[18,:]
    Cp = auxi[20,:]
    Psy = auxi[19,:]
    VPD = auxi[28,:]/10   # Check the units (should be in kPa)
    ras = auxi[21,:]
    rac = auxi[22,:]
    raa = auxi[23,:]
    rss = auxi[16,:]
    PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    rsc = np.exp(rsc_pre)  # y = log(rsc)
    rsc = np.clip(rsc,1e-5,5000)
    PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    R_s = (delta + Psy)*ras + Psy*rss
    R_c = (delta + Psy)*rac + Psy*rsc
    R_a = (delta + Psy)*raa
    w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    
    
    T_pre = w_c*PM_c
    LE_pre = w_s*PM_s + w_c*PM_c
    
    return T_pre


In [ ]:
#pip install tensorflow --upgrade
from my_functions_upgraded import *
import gc
import mpl_scatter_density
import math
import h5py
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
print(tf.__version__)
import matplotlib
# matplotlib.use('Agg')
from matplotlib import pyplot as plt
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
import pandas as pd
import argparse
import datetime
import matplotlib.dates as mdate
from sklearn.model_selection import train_test_split
import os
import numpy.ma as ma
#---- Basic settings ----
sites = os.listdir('../../Input_Data/FluxData/')
sites = [x[0:6] for x in sites]
sites = np.unique(sites)
sites.sort()
#sites = sites[1:2]
opt_hyp_params_info = pd.read_csv('Docs/hyperparameters.csv')

var_names_long = ['DateTime','TA_F_MDS','SW_IN_F_MDS','fPAR','VPD_F_MDS','WS','RH','CO2_F_MDS','LAI','NETRAD','G_F_MDS',\
                  'hc','USTAR','GPP_NT_VUT_REF','theta','theta_2','LE_F_MDS','LE_c','H_F_MDS','H_c','QCflag', 'T_ET_TEA',\
                  'T_ET_uWUE','T_ET_uWUE_BC','LE_SW_emp_calib_LE','Tr_SW_emp_calib_LE','LE_SW_emp_calib_with_TEA',\
                  'Tr_SW_emp_calib_with_TEA','LE_SW_emp_calib_with_uWUE','Tr_SW_emp_calib_with_uWUE',\
                  'LE_SW_emp_calib_with_Liuyang','Tr_SW_emp_calib_with_Liuyang',
                  'rss', 'delta','rou', 'Psy', 'Cp', 'ras', 'rac', 'raa', 'T_ET_Liuyang_2022',\
                 'LE_SW_hydr_calib_LE','Tr_SW_hydr_calib_LE',\
                   'LE_SW_hydr_calib_with_TEA','Tr_SW_hydr_calib_with_TEA','LE_SW_hydr_calib_with_uWUE',\
                   'Tr_SW_hydr_calib_with_uWUE','LE_SW_hydr_calib_with_Liuyang','Tr_SW_hydr_calib_with_Liuyang']
var_names_short = ['DateTime','TA','SRad','fPAR','VPD','WS','RH','CO2','LAI','Rn','G','hc','USTAR','GPP','theta','theta_2','LE','LE_c','H',\
                   'H_c','QCflag','T_ET_TEA','T_ET_uWUE','T_ET_uWUE_BC','LE_SW_emp_calib_LE','Tr_SW_emp_calib_LE',\
                   'LE_SW_emp_calib_with_TEA','Tr_SW_emp_calib_with_TEA','LE_SW_emp_calib_with_uWUE','Tr_SW_emp_calib_with_uWUE',\
                   'LE_SW_emp_calib_with_Liuyang','Tr_SW_emp_calib_with_Liuyang',\
                   'rss', 'delta','rou', 'Psy', 'Cp',\
                   'ras','rac', 'raa', 'T_ET_Liuyang_2022','LE_SW_hydr_calib_LE','Tr_SW_hydr_calib_LE',\
                   'LE_SW_hydr_calib_with_TEA','Tr_SW_hydr_calib_with_TEA','LE_SW_hydr_calib_with_uWUE',\
                   'Tr_SW_hydr_calib_with_uWUE','LE_SW_hydr_calib_with_Liuyang','Tr_SW_hydr_calib_with_Liuyang']
X_vars = ['Rn','G','TA','SRad','fPAR','USTAR','VPD','WS','CO2','LAI','theta'] # X variables for ML model
Y_var = ['LE']       # Target for ML
Auxi_Vars = ['LE','LE_c','H','H_c','QCflag','T_ET_TEA','T_ET_uWUE','T_ET_uWUE_BC',\
             'LE_SW_emp_calib_LE','Tr_SW_emp_calib_LE','LE_SW_emp_calib_with_TEA','Tr_SW_emp_calib_with_TEA',\
             'LE_SW_emp_calib_with_uWUE','Tr_SW_emp_calib_with_uWUE',\
             'LE_SW_emp_calib_with_Liuyang','Tr_SW_emp_calib_with_Liuyang',\
             'rss','delta','rou','Psy','Cp','ras','rac','raa','T_ET_Liuyang_2022','Rn','G','LAI','VPD',\
            'LE_SW_hydr_calib_LE','Tr_SW_hydr_calib_LE',\
                   'LE_SW_hydr_calib_with_TEA','Tr_SW_hydr_calib_with_TEA','LE_SW_hydr_calib_with_uWUE',\
                   'Tr_SW_hydr_calib_with_uWUE','LE_SW_hydr_calib_with_Liuyang','Tr_SW_hydr_calib_with_Liuyang'] # Additional variables

df_stats_LE_TEA_uWUE_EB = []
df_stats_Liuyang = []
for site in sites:
    print("----" + site + "----")
    try:
        #----Import Data (Prepare data for model training and validation----
        X_train_temp = pd.read_csv('../../Input_Data/Train_Val_Data/X_train_temp_'+site+'.csv')
        X_val_temp = pd.read_csv('../../Input_Data/Train_Val_Data/X_val_temp_'+site+'.csv')
        Y_train = pd.read_csv('../../Input_Data/Train_Val_Data/Y_train_'+site+'.csv')
        Y_val = pd.read_csv('../../Input_Data/Train_Val_Data/Y_val_'+site+'.csv')
        
        temp = pd.concat([X_train_temp,Y_train])
        temp = temp.dropna(axis=0, how='any')
        X_train_temp=temp[X_vars+Auxi_Vars]
        X_train_temp = X_train_temp.loc[:,~X_train_temp.columns.duplicated()].copy() 
        Y_train = temp[Y_var]
        
        temp = pd.concat([X_val_temp,Y_val])
        temp = temp.dropna(axis=0, how='any')
        X_val_temp=temp[X_vars+Auxi_Vars]
        X_val_temp = X_val_temp.loc[:,~X_val_temp.columns.duplicated()].copy() 
        Y_val = temp[Y_var]
        
        X_train = X_train_temp[X_vars]
        X_val = X_val_temp[X_vars]
        X_auxi_train = X_train_temp[Auxi_Vars]
        X_auxi_val = X_val_temp[Auxi_Vars]
        # Backup dataframe
        X_train_back = X_train
        X_val_back = X_val
        X_test = X_val
        Y_test = Y_val
        Y_train = np.asarray(Y_train.T)
        Y_val = np.asarray(Y_val.T)
        Y_test = np.asarray(Y_test.T)
        X_train = np.asarray(X_train.T)
        X_val = np.asarray(X_val.T)
        X_test = np.asarray(X_test.T)
        # ----- Normalization of the Input Data ----
        #var_names = {'TA','SRad','VPD','WS','RH','CO2','LAI','Rn','G','GPP','SM'}
        # Normalization
        mean = X_train.T.astype('float').mean(axis=0)
        std = X_train.T.astype('float').std(axis=0)
        X_train = ((X_train.T - mean) / std).T
        X_val = ((X_val.T - mean) / std).T
        X_test  = ((X_test.T - mean) / std).T
        
        X_train[8,:][np.isnan(X_train[8,:])] = 0  # CO2
        X_val[8,:][np.isnan(X_val[8,:])] = 0  # CO2
        X_test[8,:][np.isnan(X_test[8,:])] = 0  # CO2

        X_auxi_train = np.asarray(X_auxi_train.T)
        X_auxi_val = np.asarray(X_auxi_val.T)
        X_auxi_test = X_auxi_val
        print('Data preparation Done!')
        #------------------ ML model training-------------------------------------
        # Basic settings
        model_num = 1  # Larget LE   # <---- Change here
        learning_rate = 0.001
        minibatch_size = 64
        print_cost = True
        num_epochs = opt_hyp_params_info['epoch'][opt_hyp_params_info['Site']==site].item()
        n_hidden = opt_hyp_params_info['n_hidden'][opt_hyp_params_info['Site']==site].item()
        n_neurons = opt_hyp_params_info['n_neuron'][opt_hyp_params_info['Site']==site].item()
        #---- Training ---- (comment few lines below if already have trained parameters saved)
        parameters, _, _, _, _, _ = model_train_tuning(X_train,Y_train,X_val,Y_val,X_test,Y_test,X_auxi_train,X_auxi_val,X_auxi_test,
                                                         learning_rate,num_epochs,minibatch_size,print_cost,n_hidden,n_neurons,model_num)
        ## Save trained parameters
        import pickle
        pickle.dump(parameters, open('Trained_parameters/' + site + '_trained_ANN_params.pkl', 'wb'))

        ## Load parameters from saved pickle file
        ##parameters = pickle.load(open('Trained_parameters/' + site + '_trained_ANN_params.pkl', 'rb'))
        ##---- Prediction ----(Training)   
        #rsc_pred_train = predict_tuning(X_train,n_hidden, parameters)[0]
        #LE_pred_train = SW_model_LE_np(X_auxi_train.astype('float'), rsc_pred_train.astype('float'))
        #LE_pred_train_back = LE_pred_train
        #T_pred_train = SW_model_T_np(X_auxi_train.astype('float'), rsc_pred_train.astype('float'))
        #
        #LE_true_train = Y_train[0].astype('float')
        #
        #T_ET_TEA = X_auxi_train[5,:]
        #T_ET_TEA = np.where(T_ET_TEA == -9999, np.nan, T_ET_TEA)
        #T_true_train_TEA = LE_true_train*T_ET_TEA
        #
        #T_ET_uWUE = X_auxi_train[7,:]
        #T_ET_uWUE = np.where(T_ET_uWUE == -9999, np.nan, T_ET_uWUE)
        #T_true_train_uWUE = LE_true_train*T_ET_uWUE
        #
        #T_ET_Liuyang = X_auxi_train[24,:]
        #T_ET_Liuyang = np.where(T_ET_Liuyang == -9999, np.nan, T_ET_Liuyang)
        #T_true_train_Liuyang = LE_true_train*T_ET_Liuyang
        #
        #LE_true_train, LE_pred_train = drop_outliers2(LE_true_train, LE_pred_train)
        #
        #temp = T_pred_train
        #
        #T_true_train_TEA, T_pred_train_TEA = drop_outliers2(T_true_train_TEA, temp)
        #
        #T_true_train_uWUE, T_pred_train_uWUE = drop_outliers2(T_true_train_uWUE, temp)
        #
        #T_true_train_Liuyang, T_pred_train_Liuyang = drop_outliers2(T_true_train_Liuyang, temp)
        #
        ##---- Prediction ----(Validation)   
        #rsc_pred_val = predict_tuning(X_val,n_hidden, parameters)[0]
        #LE_pred_val = SW_model_LE_np(X_auxi_val.astype('float'), rsc_pred_val.astype('float'))
        #LE_pred_val_back = LE_pred_val
        #T_pred_val = SW_model_T_np(X_auxi_val.astype('float'), rsc_pred_val.astype('float'))
        #
        #LE_true_val = Y_val[0].astype('float')
        #
        #T_ET_TEA = X_auxi_val[5,:]
        #T_ET_TEA = np.where(T_ET_TEA == -9999, np.nan, T_ET_TEA)
        #T_true_val_TEA = LE_true_val*T_ET_TEA
        #
        #T_ET_uWUE = X_auxi_val[7,:]
        #T_ET_uWUE = np.where(T_ET_uWUE == -9999, np.nan, T_ET_uWUE)
        #T_true_val_uWUE = LE_true_val*T_ET_uWUE
        #
        #T_ET_Liuyang = X_auxi_val[24,:]
        #T_ET_Liuyang = np.where(T_ET_Liuyang == -9999, np.nan, T_ET_Liuyang)
        #T_true_val_Liuyang = LE_true_val*T_ET_Liuyang
        #
        #LE_true_val, LE_pred_val = drop_outliers2(LE_true_val, LE_pred_val)
        #
        #temp = T_pred_val
        #
        #T_true_val_TEA, T_pred_val_TEA = drop_outliers2(T_true_val_TEA, temp)
        #
        #T_true_val_uWUE, T_pred_val_uWUE = drop_outliers2(T_true_val_uWUE, temp)
        #
        #T_true_val_Liuyang, T_pred_val_Liuyang = drop_outliers2(T_true_val_Liuyang, temp)
        #
        ##--------Create Plots---------
        ##----Plot (LE)----
        ## define the fonts which are used in plot
        #font1 = {'size':16}
        #font2 = {'size':16,'color':'k'}
        #fig = plt.figure()
        #fig.set_size_inches(16,6)
        #
        ## Training
        #ax = fig.add_subplot(1, 2, 1, projection='scatter_density')
        #density =ax.scatter_density(LE_pred_train, LE_true_train,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True LE (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred LE (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30))  
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(LE_true_train, LE_pred_train)
        #stat_temp1 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(a)', font2)
        #plt.title('Training' + ' (' + site + ')', font1)
        #
        ## Validation
        #ax = fig.add_subplot(1, 2, 2, projection='scatter_density')
        #density =ax.scatter_density(LE_pred_val, LE_true_val,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True LE (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred LE (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30)) 
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(LE_true_val, LE_pred_val)
        #stat_temp2 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(b)', font2)
        #plt.title('Validation' + ' (' + site + ')', font1)
        #fig.savefig('Figures/' + site + '_Fig1_LE_comparison.png',dpi=600)
        #
        ##----Plot (T [TEA])----
        ## define the fonts which are used in plot
        #font1 = {'size':16}
        #font2 = {'size':16,'color':'k'}
        #fig = plt.figure()
        #fig.set_size_inches(16,6)
        #
        ## Training
        #ax = fig.add_subplot(1, 2, 1, projection='scatter_density')
        #density =ax.scatter_density(T_pred_train_TEA, T_true_train_TEA,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True T [TEA] (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred T (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30))  
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(T_true_train_TEA, T_pred_train_TEA)
        #stat_temp3 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(a)', font2)
        #plt.title('Training' + ' (' + site + ')', font1)
        #
        ## Validation
        #ax = fig.add_subplot(1, 2, 2, projection='scatter_density')
        #density =ax.scatter_density(T_pred_val_TEA, T_true_val_TEA,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True T [TEA] (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred LE (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30)) 
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(T_true_val_TEA, T_pred_val_TEA)
        #stat_temp4 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(b)', font2)
        #plt.title('Validation' + ' (' + site + ')', font1)
        #fig.savefig('Figures/' + site + '_Fig2_T_TEA_comparison.png',dpi=600)
        #
        ##----Plot (T [uWUE])----
        ## define the fonts which are used in plot
        #font1 = {'size':16}
        #font2 = {'size':16,'color':'k'}
        #fig = plt.figure()
        #fig.set_size_inches(16,6)
        #
        ## Training
        #ax = fig.add_subplot(1, 2, 1, projection='scatter_density')
        #density =ax.scatter_density(T_pred_train_uWUE, T_true_train_uWUE,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True T [uWUE] (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred T (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30))  
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(T_true_train_uWUE, T_pred_train_uWUE)
        #stat_temp5 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(a)', font2)
        #plt.title('Training' + ' (' + site + ')', font1)
        #
        ## Validation
        #ax = fig.add_subplot(1, 2, 2, projection='scatter_density')
        #density =ax.scatter_density(T_pred_val_uWUE, T_true_val_uWUE,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True T [uWUE] (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred LE (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30)) 
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(T_true_val_uWUE, T_pred_val_uWUE)
        #stat_temp6 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(b)', font2)
        #plt.title('Validation' + ' (' + site + ')', font1)
        #fig.savefig('Figures/' + site + '_Fig3_T_uWUE_comparison.png',dpi=600)
        #
        ##----Energy Balance----
        ## Train
        #Rn_G = X_auxi_train[25,:] - X_auxi_train[26,:]
        #H_LE = X_auxi_train[2,:] + LE_pred_train_back
        #Rn_G, H_LE = drop_outliers2(Rn_G, H_LE)
        #
        #font1 = {'size':16}
        #font2 = {'size':16,'color':'k'}
        #fig = plt.figure()
        #fig.set_size_inches(16,6)
#
        #ax = fig.add_subplot(1, 2, 1, projection='scatter_density')
        #density =ax.scatter_density(Rn_G, H_LE,cmap=plt.cm.jet)
        #ax.set_xlim(0, 1000)
        #ax.set_ylim(0, 1000)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('H + LE (W $m^{-2}$)',font1)
        #ax.set_xlabel('$R_{n}$ - G (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30))  
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test(H_LE, Rn_G)
        #stat_temp7 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(50,1050,'(a)', font2)
        #plt.title('Training' + ' (' + site + ')', font1)
        #
        ## Val
        #Rn_G = X_auxi_val[25,:] - X_auxi_val[26,:]
        #H_LE = X_auxi_val[2,:] + LE_pred_val_back
        #Rn_G, H_LE = drop_outliers2(Rn_G, H_LE)
#
        #ax = fig.add_subplot(1, 2, 2, projection='scatter_density')
        #density =ax.scatter_density(Rn_G, H_LE,cmap=plt.cm.jet)
        #ax.set_xlim(0, 1000)
        #ax.set_ylim(0, 1000)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('H + LE (W $m^{-2}$)',font1)
        #ax.set_xlabel('$R_{n}$ - G (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30))  
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test(H_LE, Rn_G)
        #stat_temp8 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(50,1050,'(b)', font2)
        #plt.title('Validation' + ' (' + site + ')', font1)
        #fig.savefig('Figures/' + site + '_Fig4_EB.png',dpi=600)
        #
        #df_temp = [stat_temp1, stat_temp2, stat_temp3, stat_temp4, stat_temp5, stat_temp6, stat_temp7, stat_temp8]
        #df_temp = pd.DataFrame(df_temp, columns=['A1','rmse_value','MAPE_value','R2','num0','MAE_value'])
        #df_temp = np.around(df_temp,2)
        #df_temp['Site'] = site
        #df_temp.index = ['LE_train','LE_val','T_TEA_train','T_TEA_val','T_uWUE_train','T_uWUE_val',
        #                'EB_train','EB_val']
        #df_stats_LE_TEA_uWUE_EB.append(df_temp) 
#
        ##----Plot (T [Liuyang])----
        ## define the fonts which are used in plot
        #font1 = {'size':16}
        #font2 = {'size':16,'color':'k'}
        #fig = plt.figure()
        #fig.set_size_inches(16,6)
        #
        ## Training
        #ax = fig.add_subplot(1, 2, 1, projection='scatter_density')
        #density =ax.scatter_density(T_pred_train_Liuyang, T_true_train_Liuyang,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True T [Liuyang_2022] (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred T (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30))  
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(T_true_train_Liuyang, T_pred_train_Liuyang)
        #stat_temp9 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(a)', font2)
        #plt.title('Training' + ' (' + site + ')', font1)
        #
        ## Validation
        #ax = fig.add_subplot(1, 2, 2, projection='scatter_density')
        #density =ax.scatter_density(T_pred_val_Liuyang, T_true_val_Liuyang,cmap=plt.cm.jet)
        #ax.set_xlim(0, 800)
        #ax.set_ylim(0, 800)
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        #ax.set_ylabel('True T [Liuyang_2022] (W $m^{-2}$)',font1)
        #ax.set_xlabel('Pred LE (W $m^{-2}$)',font1)
        #density.cmap.set_under('white')
        #density.set_clim(vmin=1e-5, vmax=30)
        #cb=plt.colorbar(density)
        #cb.set_ticks([5, 10, 15, 20, 25, 30])
        #cb.set_ticklabels((5, 10, 15, 20, 25, 30)) 
        #cb.ax.tick_params(labelsize=16)
        #_ = plt.plot([0, 5000], [0, 5000],'k--')
        #A1,rmse_value,MAPE_value,R2,num0,MAE_value = plot_test_LE_T(T_true_val_Liuyang, T_pred_val_Liuyang)
        #stat_temp10 = [A1,rmse_value,MAPE_value,R2,num0,MAE_value]
        #plt.text(25,825,'(b)', font2)
        #plt.title('Validation' + ' (' + site + ')', font1)
        #fig.savefig('Figures/' + site + '_Fig3_T_Liuyang_comparison.png',dpi=600)
        #
        #df_temp_L = [stat_temp9, stat_temp10]
        #df_temp_L = pd.DataFrame(df_temp_L, columns=['A1','rmse_value','MAPE_value','R2','num0','MAE_value'])
        #df_temp_L = np.around(df_temp_L,2)
        #df_temp_L['Site'] = site
        #df_temp_L.index = ['T_Liuyang_train','T_Liuyang_val']
        #df_stats_Liuyang.append(df_temp_L)
        
    except (OSError,ValueError):
        pass
    
        